### Load environment variables

In [1]:
import os
import sys  
sys.path.append("..")  

from dotenv import load_dotenv
_ = load_dotenv('.env')

# This lines should be modified according to your environment variables
os.environ['OPENAI_API_TYPE'] =  os.getenv('OPENAI_API_TYPE')
os.environ['OPENAI_API_VERSION'] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ['OPENAI_API_BASE'] = os.getenv("AZURE_OPENAI_API_ENDPOINT")
os.environ['OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ['LANGCHAIN_HUB_API_KEY'] = os.getenv("LANG_SMITH_API_KEY")  ## This is not necessary. You can pull without this.
LLM_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_NAME")

### Get a random persona

In [2]:
from user_persona import UserPersona, get_random_user_persona
persona = get_random_user_persona().json()
persona

'{"gender": "female", "name": "Emily Mckinney", "language": "English", "nationality": "China", "age": 48, "hobbies": ["yoga", "skateboarding", "astronomy", "knitting"], "talkative": false, "characteristics": ["NotVibrant", "NotKind_hearted"], "education_level": "phd_degree_level"}'

### Fetch the prompt and create a chain (using LCEL)

In [3]:
from langchain import hub
from langchain.chat_models import AzureChatOpenAI

prompt = hub.pull("jet-taekyo-lee/persona-based-dialog")
chat_model = AzureChatOpenAI(deployment_name=LLM_DEPLOYMENT_NAME, temperature=0)

def streaming_output(response_generator):
    for chunk in response_generator:
        print(chunk.content, end='')    

chain = prompt | chat_model

### Get a response with an empty input (Initiating a conversation)

In [4]:
response = chain.stream({'persona':persona, 'input': ''})
streaming_output(response)

## Persona Analysis
- **Characteristics**: NotVibrant, NotKind_hearted
- **Other Attributes**: Female, named Emily Mckinney, speaks English, Chinese nationality, 48 years old, hobbies include yoga, skateboarding, astronomy, and knitting, not talkative, has a PhD degree.

## Conflict Detection in persona
- **Conflict 1**: The name 'Emily Mckinney' is not typically Chinese. It's more common in English-speaking countries.
- **Conflict 2**: The hobbies 'skateboarding' and 'yoga' seem to contradict the 'NotVibrant' characteristic.

## Conflict Resolution in persona
- **Adjusted Characteristics**: Changed the name to 'Mei Ling' which is a common name in China and sounds similar to 'Emily Mckinney'. Removed 'skateboarding' from hobbies as it's a vibrant activity.

## Detailed Persona Description
Mei Ling is a 48-year-old Chinese woman who is not particularly vibrant or kind-hearted. She is not very talkative and prefers to keep to herself. Despite her quiet nature, she has a variety of hobbie

In [5]:
response = chain.stream({'persona':persona, 'input': 'Life is so boring. Why is that?'})
streaming_output(response)

## Persona Analysis
- **Characteristics**: NotVibrant, NotKind_hearted
- **Other Attributes**: Female, named Emily Mckinney, speaks English, Chinese nationality, 48 years old, hobbies include yoga, skateboarding, astronomy, knitting, not talkative, has a PhD degree.

## Conflict Detection in persona
- **Conflict 1**: The name "Emily Mckinney" is not typically Chinese. It's more common in English-speaking countries.
- **Conflict 2**: The hobbies of yoga, skateboarding, astronomy, and knitting seem to contradict the characteristic of "NotVibrant". These hobbies are usually associated with vibrant and active individuals.

## Conflict Resolution in persona
- **Adjusted Characteristics**: NotVibrant, NotKind_hearted
- **Adjusted Name**: Emily Mckinney, but she was born and raised in an English-speaking country and later moved to China.

## Detailed Persona Description
Emily Mckinney is a 48-year-old woman of Chinese nationality, though her name suggests she might have been born and raised i